# Merging Our Data
Raveesha Gupta
Sarita Sapkota
Max Silverstein
December 14, 2017

Run this notebook after merging the data.

In [1]:
#Load our packages
! conda install geopandas -yq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point, Polygon

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold



# All requested packages already installed.
# packages in environment at /opt/conda:
#
geopandas                 0.3.0                    py36_0    conda-forge


In [2]:
#Change Directory
import os
os.chdir('Project Files')

#Import state abbreviations dictionary
from us_state_abbrev import *

In [26]:
#2000 census geographic area, note that the header is on line 1
data1 = pd.read_csv('DEC_00_SF1_GCTPH1.US05PR_with_ann.csv', header = 1, encoding='latin-1')
data1 = data1.rename(columns={'Geographical Area': 'CTYNAME'})
data1.CTYNAME

0                                    United States
1                          United States - Alabama
2         United States - Alabama - Autauga County
3         United States - Alabama - Baldwin County
4         United States - Alabama - Barbour County
5            United States - Alabama - Bibb County
6          United States - Alabama - Blount County
7         United States - Alabama - Bullock County
8          United States - Alabama - Butler County
9         United States - Alabama - Calhoun County
10       United States - Alabama - Chambers County
11       United States - Alabama - Cherokee County
12        United States - Alabama - Chilton County
13        United States - Alabama - Choctaw County
14         United States - Alabama - Clarke County
15           United States - Alabama - Clay County
16       United States - Alabama - Cleburne County
17         United States - Alabama - Coffee County
18        United States - Alabama - Colbert County
19        United States - Alaba

In [27]:
#reformat CTYNAME to our common format
temp=data1.CTYNAME.str.split("-", 2, expand=True)
temp[1] = temp[1].replace(us_state_abbrev, regex=True).str.strip()
temp[2] = temp[2].str.strip()
data1.CTYNAME = temp[1] + "-" + temp[2]
data1 = data1[data1.CTYNAME.notnull()].reset_index()
data1.CTYNAME


0           AL-Autauga County
1           AL-Baldwin County
2           AL-Barbour County
3              AL-Bibb County
4            AL-Blount County
5           AL-Bullock County
6            AL-Butler County
7           AL-Calhoun County
8          AL-Chambers County
9          AL-Cherokee County
10          AL-Chilton County
11          AL-Choctaw County
12           AL-Clarke County
13             AL-Clay County
14         AL-Cleburne County
15           AL-Coffee County
16          AL-Colbert County
17          AL-Conecuh County
18            AL-Coosa County
19        AL-Covington County
20         AL-Crenshaw County
21          AL-Cullman County
22             AL-Dale County
23           AL-Dallas County
24           AL-DeKalb County
25           AL-Elmore County
26         AL-Escambia County
27           AL-Etowah County
28          AL-Fayette County
29         AL-Franklin County
                ...          
3111       WI-Washburn County
3112     WI-Washington County
3113      

In [28]:
#create new dataframe, duplicate it for each subsequent year
area = data1[['CTYNAME', 'Area in square miles - Land area']]
censusarea00 = pd.DataFrame()
for x in range(2005, 2011):
    temp = area.copy()
    temp['year'] = str(x)
    censusarea00 = censusarea00.append(temp)

In [29]:
#Census population estimates read
popnum00 = pd.read_csv('co-est2010-alldata.csv', encoding='latin-1')

In [30]:
#reformat the county name to our preferred format
popnum00.STNAME = popnum00.STNAME.replace(us_state_abbrev, regex=True)
popnum00.CTYNAME = popnum00.STNAME + "-" + popnum00.CTYNAME


In [31]:
#reformat so that each year isn't its own set of variables
basepop00 = popnum00[['CTYNAME','CENSUS2000POP']].copy()
population00 = pd.DataFrame()
for x in range(2005, 2011):
    temporary = basepop00.copy()
    year = str(x)
    for y in popnum00.columns:
        if y.endswith(year):
            label = y.split(year)[0]
            temporary[label] = popnum00[y]
    temporary['year'] = year
    population00 = population00.append(temporary)

In [32]:
population00.to_csv('population00.csv')

In [33]:
popdensity00 = population00.merge(censusarea00, how='left', on=['year','CTYNAME'])

In [34]:
#2010 census geographic area, note that the header is on line 1
data = pd.read_csv('DEC_10_SF1_GCTPH1.ST05_with_ann.csv', header = 1, encoding='latin-1')
data = data.rename(columns={'Geographic area': 'CTYNAME'})

In [35]:
#reformat CTYNAME to our common format
temp=data.CTYNAME.str.split("-", 1, expand=True)
temp[0] = temp[0].replace(us_state_abbrev, regex=True).str.strip()
temp[1] = temp[1].str.strip()
data.CTYNAME = temp[0] + "-" + temp[1]
data = data[data.CTYNAME.notnull()].reset_index()
data.CTYNAME

0                         AL-Autauga County
1                         AL-Baldwin County
2                         AL-Barbour County
3                            AL-Bibb County
4                          AL-Blount County
5                         AL-Bullock County
6                          AL-Butler County
7                         AL-Calhoun County
8                        AL-Chambers County
9                        AL-Cherokee County
10                        AL-Chilton County
11                        AL-Choctaw County
12                         AL-Clarke County
13                           AL-Clay County
14                       AL-Cleburne County
15                         AL-Coffee County
16                        AL-Colbert County
17                        AL-Conecuh County
18                          AL-Coosa County
19                      AL-Covington County
20                       AL-Crenshaw County
21                        AL-Cullman County
22                           AL-

In [36]:
#create new dataframe, duplicate it for each subsequent year
area = data[['CTYNAME', 'Area in square miles - Land area']]
censusarea = pd.DataFrame()
for x in range(2010, 2018):
    temp = area.copy()
    temp['year'] = str(x)
    censusarea = censusarea.append(temp)


In [37]:
#Census population estimates read
popnum = pd.read_csv('co-est2016-alldata.csv', encoding='latin-1')

In [38]:
#reformat the county name to our preferred format
popnum.STNAME = popnum.STNAME.replace(us_state_abbrev, regex=True)
popnum.CTYNAME = popnum.STNAME + "-" + popnum.CTYNAME

In [39]:
#reformat so that each year isn't its own set of variables
basepop = popnum[['CTYNAME','CENSUS2010POP']].copy()
population = pd.DataFrame()
for x in range(2010, 2017):
    temporary = basepop.copy()
    year = str(x)
    for y in popnum.columns:
        if y.endswith(year):
            label = y.split(year)[0]
            temporary[label] = popnum[y]
    temporary['year'] = year
    population = population.append(temporary)

In [81]:
#Merge census area with population data, NEED TO CALCULATE DENSITY
popdensity10 = population.merge(censusarea, how='left', on=['year','CTYNAME'])

In [82]:
popdensity = popdensity00.append(popdensity10)
popdensity['density'] = popdensity['POPESTIMATE']/popdensity['Area in square miles - Land area']
popdensity = popdensity.loc[:, ['CTYNAME', 'year', 'density', 'POPESTIMATE']]
popdensity.year = popdensity.year.astype('float64')
popdensity.CTYNAME = popdensity.CTYNAME.str.replace(' County', '')
popdensity.CTYNAME

0            AL-Alabama
1            AL-Autauga
2            AL-Baldwin
3            AL-Barbour
4               AL-Bibb
5             AL-Blount
6            AL-Bullock
7             AL-Butler
8            AL-Calhoun
9           AL-Chambers
10          AL-Cherokee
11           AL-Chilton
12           AL-Choctaw
13            AL-Clarke
14              AL-Clay
15          AL-Cleburne
16            AL-Coffee
17           AL-Colbert
18           AL-Conecuh
19             AL-Coosa
20         AL-Covington
21          AL-Crenshaw
22           AL-Cullman
23              AL-Dale
24            AL-Dallas
25            AL-DeKalb
26            AL-Elmore
27          AL-Escambia
28            AL-Etowah
29           AL-Fayette
              ...      
22321     WI-Washington
22322       WI-Waukesha
22323        WI-Waupaca
22324       WI-Waushara
22325      WI-Winnebago
22326           WI-Wood
22327        WY-Wyoming
22328         WY-Albany
22329       WY-Big Horn
22330       WY-Campbell
22331         WY

In [64]:
bankruptcy = pd.read_csv('bankruptcy.csv', header = 0, encoding='latin-1')
bankruptcy = bankruptcy.rename(columns={'county': 'CTYNAME'})
bankruptcy.CTYNAME = bankruptcy.CTYNAME.str.replace(' end', '')

In [65]:
#reformat CTYNAME to our common format
bankruptcy.CTYNAME = bankruptcy.CTYNAME.str.strip()
temp = bankruptcy.CTYNAME.str.rsplit(" ", 1, expand=True)
temp[1] = temp[1].replace(us_state_abbrev, regex=True).str.strip()
bankruptcy['state'] = temp[1]
bankruptcy.CTYNAME = temp[1] + "-" + temp[0]
bankruptcy.CTYNAME
bankruptcy.to_csv('bankrupt.csv')
bankruptcy.year = bankruptcy.year.astype('float64')
bankruptcy.CTYNAME = bankruptcy.CTYNAME.str.replace(' County', '')
bankruptcy = bankruptcy[bankruptcy.CTYNAME != "Value-Median"].reset_index(drop=True)
bankruptcy = bankruptcy[bankruptcy.CTYNAME.notnull()].reset_index(drop=True)

In [66]:
bankruptcy

,CTYNAME,year,totalfilings,state
0,AL-Autauga,2005.0,424.0,AL
1,AL-Baldwin,2005.0,1380.0,AL
2,AL-Barbour,2005.0,257.0,AL
3,AL-Bibb,2005.0,212.0,AL
4,AL-Blount,2005.0,505.0,AL
5,AL-Bullock,2005.0,65.0,AL
6,AL-Butler,2005.0,167.0,AL
7,AL-Calhoun,2005.0,989.0,AL
8,AL-Chambers,2005.0,435.0,AL
9,AL-Cherokee,2005.0,168.0,AL


In [67]:
cvdeath = pd.read_csv('Cardiovascular_death_per_100000.csv', header = 0, encoding='latin-1')
cvdeath = cvdeath.rename(columns={'County': 'CTYNAME'})

In [68]:
cvdeath.CTYNAME = cvdeath.State + "-" + cvdeath.CTYNAME
cvdeath.CTYNAME = cvdeath.CTYNAME.str.strip()
cvdeath.year = cvdeath.year.astype('float64')
cvdeath.CTYNAME = cvdeath.CTYNAME.str.replace(' County', '')

In [69]:
cvdeath

,year,CTYNAME,State,Cardiovascular deaths,Unnamed: 4
0,2005.0,AK-Aleutians East,AK,333.7,
1,2005.0,AK-Sitka,AK,366.8,
2,2005.0,AK-Haines,AK,377.5,
3,2005.0,AK-Juneau,AK,377.5,
4,2005.0,AK-Denali,AK,410.9,
5,2005.0,AK-Fairbanks North Star,AK,419.2,
6,2005.0,AK-Anchorage,AK,430.2,
7,2005.0,AK-Kodiak Island,AK,430.4,
8,2005.0,AK-Kenai Peninsula,AK,440.6,
9,2005.0,AK-Lake and Peninsula,AK,448.5,


In [70]:
loanaccess = pd.read_csv('loan_access.csv', header = 0, encoding = 'latin-1')
loanaccess = loanaccess.rename(columns={'county': 'CTYNAME'})
loanaccess.year = loanaccess.year.astype('float64')
loanaccess.CTYNAME = loanaccess.CTYNAME.str.replace(' County', '')
loanaccess.CTYNAME

0                          NaN
1                   AL-Autauga
2                   AL-Baldwin
3                   AL-Barbour
4                      AL-Bibb
5                    AL-Blount
6                   AL-Bullock
7                    AL-Butler
8                   AL-Calhoun
9                  AL-Chambers
10                 AL-Cherokee
11                  AL-Chilton
12                  AL-Choctaw
13                   AL-Clarke
14                     AL-Clay
15                 AL-Cleburne
16                   AL-Coffee
17                  AL-Colbert
18                  AL-Conecuh
19                    AL-Coosa
20                AL-Covington
21                 AL-Crenshaw
22                  AL-Cullman
23                     AL-Dale
24                   AL-Dallas
25                   AL-DeKalb
26                   AL-Elmore
27                 AL-Escambia
28                   AL-Etowah
29                  AL-Fayette
                 ...          
9894           VA-Emporia city
9895    

In [71]:
loanaccess.year.dtype

dtype('float64')

In [72]:
divorce = pd.read_csv('Divorce.csv', encoding = 'latin-1')
divorce = divorce.rename(columns={'county': 'CTYNAME'})
divorce['Divorced; Estimate; Population 15 years and over'] = divorce['Divorced; Estimate; Population 15 years and over'].str.replace('N', '')

In [73]:
#reformat CTYNAME to our common format
temp=divorce.CTYNAME.str.split(",", 1, expand=True)
temp[1] = temp[1].replace(us_state_abbrev, regex=True).str.strip()
temp[0] = temp[0].str.strip()
divorce.CTYNAME = temp[1] + "-" + temp[0]
divorce.year = divorce.year.astype('float64')
divorce.CTYNAME = divorce.CTYNAME.str.replace(' County', '')
divorce.CTYNAME

0                             AL-Baldwin
1                             AL-Calhoun
2                             AL-Cullman
3                              AL-DeKalb
4                              AL-Elmore
5                              AL-Etowah
6                             AL-Houston
7                           AL-Jefferson
8                          AL-Lauderdale
9                                 AL-Lee
10                          AL-Limestone
11                            AL-Madison
12                           AL-Marshall
13                             AL-Mobile
14                         AL-Montgomery
15                             AL-Morgan
16                          AL-St. Clair
17                             AL-Shelby
18                          AL-Talladega
19                         AL-Tuscaloosa
20                             AL-Walker
21             AK-Anchorage Municipality
22       AK-Fairbanks North Star Borough
23          AK-Matanuska-Susitna Borough
24              

In [74]:
unemployment = pd.read_csv('Unemployment.csv', encoding = 'latin-1')
unemployment = unemployment.rename(columns={'county': 'CTYNAME', 'Year': 'year'})
unemployment.year = unemployment.year.astype('float64')

In [75]:
#reformat CTYNAME to our common format
temp=unemployment.CTYNAME.str.split(",", 1, expand=True)
temp[1] = temp[1].replace(us_state_abbrev, regex=True).str.strip()
temp[0] = temp[0].str.strip()
unemployment.CTYNAME = temp[1] + "-" + temp[0]
unemployment.CTYNAME = unemployment.CTYNAME.str.replace(' County', '')
unemployment.CTYNAME

0                            AL-Baldwin
1                            AL-Calhoun
2                            AL-Cullman
3                             AL-DeKalb
4                             AL-Elmore
5                             AL-Etowah
6                            AL-Houston
7                          AL-Jefferson
8                         AL-Lauderdale
9                                AL-Lee
10                         AL-Limestone
11                           AL-Madison
12                          AL-Marshall
13                            AL-Mobile
14                        AL-Montgomery
15                            AL-Morgan
16                         AL-St. Clair
17                            AL-Shelby
18                         AL-Talladega
19                        AL-Tuscaloosa
20                            AL-Walker
21            AK-Anchorage Municipality
22      AK-Fairbanks North Star Borough
23         AK-Matanuska-Susitna Borough
24                            AZ-Apache


In [76]:
files = os.listdir()
files_xls = [f for f in files if f[:8] == 'pa_reimb']   
reimb = pd.DataFrame()
for f in files_xls:
    year = f[-8:-4]
    x = pd.read_excel(f, header=0)
    name = list(filter(lambda x: x.startswith('Medicare enrollees'), x.columns.get_level_values(0)))
    x = x.rename(columns={'County name': 'CTYNAME', name[0]: 'Medicare enrollees '})
    for y in x.columns:
        if y.endswith(year + ')'):
            label = y.split('(' + year)[0]
            x.rename(columns = {y : label} , inplace=True)
    x['year'] = year        
    reimb = reimb.append(x)
    # if reimb.empty:
       # reimb = reimb.append(x)
    #else :
      #  reimb = reimb.merge(x, how='left', on='County name')


#a = reimb.describe()
#print (a)
reimb['Index'] = reimb.index
reimb = reimb.sort_values(['year', 'Index'])
reimb.drop(labels = 'Index', axis = 1, inplace=True)       
reimb = reimb.loc[:, reimb.columns.get_level_values(0).isin({'CTYNAME', 'Medicare enrollees ', 'Total Medicare reimbursements per enrollee (Parts A and B) ', 'year'})]                         
reimb.year = reimb.year.astype('float64')
reimb.CTYNAME = reimb.CTYNAME.str.replace(' County', '')
reimb.to_csv('reimb.csv')

reimb2 = reimb
reimb2 = reimb2.reset_index()
reimb2.columns

Index(['index', 'CTYNAME', 'Medicare enrollees ',
       'Total Medicare reimbursements per enrollee (Parts A and B) ', 'year'],
      dtype='object')

In [83]:
mydata = bankruptcy.merge(popdensity, how='left', on=['year', 'CTYNAME'])
mydata = mydata.merge(loanaccess, how='left', on=['year', 'CTYNAME'])
mydata = mydata.merge(unemployment, how='left', on=['year', 'CTYNAME'])
mydata = mydata.merge(cvdeath, how='left', on=['year', 'CTYNAME'])
mydata = mydata.merge(divorce, how='left', on=['year', 'CTYNAME'])
mydata = mydata.merge(reimb, how='left', on=['year', 'CTYNAME'])




In [84]:
mydata.to_csv('mergeddata.csv')

In [85]:
mydata.columns

Index(['CTYNAME', 'year', 'totalfilings', 'state', 'density', 'POPESTIMATE',
       'credit_access', 'totpop', 'unemp', 'unemp_rate', 'State',
       'Cardiovascular deaths', 'Unnamed: 4',
       'Divorced; Estimate; Population 15 years and over',
       'Medicare enrollees ',
       'Total Medicare reimbursements per enrollee (Parts A and B) '],
      dtype='object')

In [80]:
reimb.columns

Index(['CTYNAME', 'Medicare enrollees ',
       'Total Medicare reimbursements per enrollee (Parts A and B) ', 'year'],
      dtype='object')